In [41]:
import torch
import librosa
import numpy as np
import os,glob

import IPython.display as ipd
# ipd.Audio('audio/conga_groove.wav') # load a local WAV file
# ipd.Audio(x, rate=sr) # load a NumPy array
import matplotlib
import matplotlib.pylab as plt
from matplotlib import cm

import soundfile as sf

In [16]:
root = '/home/data/kbh/AVTR/sample/'
root_output = './'

In [17]:
list_clean    = [x for x in glob.glob(os.path.join(root,'clean','*.wav'))]
list_hospital = [x for x in glob.glob(os.path.join(root,'hospital','*.wav'))]
list_keyboard = [x for x in glob.glob(os.path.join(root,'keyboard','*.wav'))]
list_noise    = [x for x in glob.glob(os.path.join(root,'background','*.wav'))]

print(len(list_clean))

10


In [18]:
SNR_keyboard = 5.0
SNR_hospital = 5.0
SNR_noise    = 5.0

sr      = 16000
n_fft   = 640
n_shift = 160
n_mels  = 32

sec_keyboard = 5.0
sec_hospital = 4.0
sec_clean    = 5.0
sec_noise    = 10.0

In [53]:
mel_basis = librosa.filters.mel(sr=sr, n_fft=n_fft, n_mels=n_mels)

for idx in range(len(list_clean)) : 
#for idx in [0]:
    raw_keyboard,_  = librosa.load(list_keyboard[idx],sr=sr)
    raw_hospital,_ = librosa.load(list_hospital[idx],sr=sr)
    raw_clean,_     = librosa.load(list_clean[idx],sr=sr)
    raw_noise,_     = librosa.load(list_noise[np.random.randint(4)],sr=sr)
    
    # Indexing
    n_sample_keyboard = int(sec_keyboard * sr)
    n_sample_hospital = int(sec_hospital * sr)
    n_sample_clean = int(sec_clean * sr)
    n_sample_noise = int(sec_noise * sr)

    idx_keyboard = np.random.randint(len(raw_keyboard) - n_sample_keyboard)
    idx_hospital = np.random.randint(len(raw_hospital) - n_sample_hospital)
    idx_clean    = np.random.randint(len(raw_clean)    - n_sample_clean)
    idx_noise    = np.random.randint(len(raw_noise)    - n_sample_noise)

    sample_keyboard = raw_keyboard[idx_keyboard : idx_keyboard + n_sample_keyboard]
    sample_hospital = raw_hospital[idx_hospital : idx_hospital + n_sample_hospital]
    sample_clean    = raw_clean   [idx_clean    : idx_clean    + n_sample_clean]
    sample_noise    = raw_noise   [idx_noise    : idx_noise    + n_sample_noise]
    
    norm_keyboard = sample_keyboard/np.max(np.abs(sample_keyboard))
    norm_hospital = sample_hospital/np.max(np.abs(sample_hospital))
    norm_clean    = sample_clean/np.max(np.abs(sample_clean))
    norm_noise    = sample_noise/np.max(np.abs(sample_noise))
    
    ## Mixing
    # SNR
    ratio_keyboard = np.power(10,SNR_keyboard/10)
    ratio_hospital = np.power(10,SNR_hospital/10)
    ratio_noise = np.power(10,SNR_noise/10)
    
    mean_energy_keyboard = np.sqrt(np.sum(np.power(norm_keyboard,2)))/n_sample_keyboard
    mean_energy_hospital = np.sqrt(np.sum(np.power(norm_hospital,2)))/n_sample_hospital
    mean_energy_clean = np.sqrt(np.sum(np.power(norm_clean,2)))/n_sample_clean
    mean_energy_noise = np.sqrt(np.sum(np.power(norm_noise,2)))/n_sample_noise

    energy_normal_keyboard = np.sqrt(mean_energy_clean)/np.sqrt(mean_energy_keyboard)
    energy_normal_hospital = np.sqrt(mean_energy_clean)/np.sqrt(mean_energy_hospital)
    energy_normal_noise = np.sqrt(mean_energy_clean)/np.sqrt(mean_energy_noise)

    weight_keyboard = energy_normal_keyboard/np.sqrt(ratio_keyboard)
    weight_hospital = energy_normal_hospital/np.sqrt(ratio_hospital)
    weight_noise    = energy_normal_noise/np.sqrt(ratio_noise)

    mix_keyboard = norm_keyboard * weight_keyboard
    mix_hospital = norm_hospital * weight_hospital
    mix_noise = norm_noise * weight_noise
    # Mix
    offset_keyboard = np.random.randint(n_sample_noise - n_sample_keyboard)
    offset_hospital = np.random.randint(n_sample_noise - n_sample_hospital)
    offset_clean    = np.random.randint(n_sample_noise - n_sample_clean)

    mixed                                                       = mix_noise
    mixed[offset_keyboard:offset_keyboard + n_sample_keyboard] += mix_keyboard
    mixed[offset_hospital:offset_hospital + n_sample_hospital] += mix_hospital
    mixed[offset_clean:offset_clean       + n_sample_clean]    += norm_clean
    mixed = mixed/np.max(np.abs(mixed))
    
    #sf.write(root_output + '/mix.wav',mixed,16000)
    
     # Wav to Mel
    spec = librosa.stft(mixed,window='hann',n_fft=n_fft,hop_length=n_shift, win_length=None,center=False)
    
    mel = np.matmul(mel_basis,np.abs(spec))
    pt = torch.from_numpy(mel)
    
    # Label (round on edge)
    label = torch.zeros(int(np.ceil(n_sample_noise/n_shift)) + 1-int(n_fft/n_shift) )
    label[int(np.round(offset_clean/n_shift)) :int(np.round((offset_clean+n_sample_clean)/n_shift)) ] = 1

    # save
    #print(pt.shape)
    offset = 100
    
    data = {"mel":pt[:,offset:offset+32],"label":label[offset:offset+32]}
    torch.save(data,os.path.join(root_output,str(idx)+'.pt'))
    
    print(data["mel"].shape)
    print(data["label"].shape)
    #print(data)
    
    #im = plt.imshow(data["mel"][:,:], cmap=cm.jet, aspect='auto')
    #plt.colorbar(im)
    #plt.clim(0.1,0.3)
    #plt.show()


torch.Size([32, 32])
torch.Size([32])
torch.Size([32, 32])
torch.Size([32])
torch.Size([32, 32])
torch.Size([32])
torch.Size([32, 32])
torch.Size([32])
torch.Size([32, 32])
torch.Size([32])
torch.Size([32, 32])
torch.Size([32])
torch.Size([32, 32])
torch.Size([32])
torch.Size([32, 32])
torch.Size([32])
torch.Size([32, 32])
torch.Size([32])
torch.Size([32, 32])
torch.Size([32])


1 sample 길이는 = 1초 / 샘플레이트  
0.010 초는?

몇 샘플?

1 샘플이 0.0000625 초  
=> shift 160 frame 640 

0 1 2 3 4 5 6 7 8 9 
0 0 0 1 1 0 0 0 0 0 => 1



In [55]:
labels = [0, 1, 0, 0, 1 , 0]
labels = [0, 0, 0, 0, 0 , 0]
label = False
for i in labels :
    label = label or i
    
print(label)

0


In [56]:
x = '123$5435.wav'
'@' in x

False

In [57]:
x = '123$5435qwe@@.wav'
'@' in x

True

In [66]:
np.random.rand(1)*5

array([2.92696859])